pip install selenium

### Função pra transformar a lista de infos na string a se renomear os arquivos

In [1]:
def formatar_string(list_info, uregi):
    
    meta = list_info[4][17:20]
    município = list_info[2]
    data = list_info[7].split('/')
    visita = list_info[5]
    
    if(meta == "1.1"):
        nome_do_arquivo = meta + "_" + uregi + "_" + município + "_" + "Escolas Estaduais" + "_" + data[2] + "." + data[1] + "." + data[0] + ".pdf"
    
    return nome_do_arquivo

### Função pra pegar o nome original do arquivo

In [2]:
def pegar_nome_original(directory):
    
    time.sleep(2)
    """
        Tem formas avançadas de determinar se o arquivo foi baixado. Aqui alguns links
        https://stackoverflow.com/questions/34338897/python-selenium-find-out-when-a-download-has-completed/34338926#34338926
        https://stackoverflow.com/questions/21746750/check-and-wait-until-a-file-exists-to-read-it
    """
    
    data_criacao = lambda f: f.stat().st_ctime

    files = directory.glob('*.pdf')
    nome_original = sorted(files, key=data_criacao, reverse=True)
    
    """
    Essa função vai pegar todos os pdfs da pasta e colocar o caminho deles em uma lista(nome_original) da qual eu vou pegar o
    primeiro termo que seria o mais recente criado. Então por tudo que é mais sagrado, que não tenha outros arquivos pdfs de
    outra natureza nessa pasta senão vai renomear e mover ele.
    """
    
    if not nome_original:
        nome_original.append("Alguma coisa")

    return nome_original[0] #Tem o endereço completo ou a expressão "Alguma coisa"
    
    

### Criando sufixos (n) para arquivos de mesmo nome e talvez assuntos diferentes

In [3]:
def nome_formatado_sufixo(nome_formatado, n):
    nome_formatado = nome_formatado[:-4]
    nome_formatado = nome_formatado + f" ({n})" + ".pdf"
    
    return nome_formatado

### Renomeando e movendo o arquivo baixado

In [4]:
def renomear_e_mover_arquivo(nome_formatado, nome_original, SRE, path, uregi, diretório_de_destino, meta):
    
    if(nome_original != "Alguma coisa"):
        n = 1
        
        os.rename(nome_original, nome_formatado)
        
        endereco_do_arquivo_a_ser_movido = path + nome_formatado
        
        
        arquivo_de_destino = diretório_de_destino + "\\" + nome_formatado
        my_file = Path(arquivo_de_destino)
        if my_file.is_file():
            
            while my_file.is_file():                
                
                if n == 1:
                    os.rename(nome_formatado, nome_formatado_sufixo(nome_formatado, n))
                else:
                    #nome_formatado_sufixo_menos1 = nome_formatado + f" ({n-1})"
                    os.rename(nome_formatado_sufixo(nome_formatado, n-1), nome_formatado_sufixo(nome_formatado, n))

                endereco_do_arquivo_a_ser_movido = path + nome_formatado_sufixo(nome_formatado, n)
                
                arquivo_de_destino = diretório_de_destino + "\\" + nome_formatado_sufixo(nome_formatado, n)
                my_file = Path(arquivo_de_destino)
                
                if my_file.is_file():
                    n += 1
                else:
                    shutil.move(endereco_do_arquivo_a_ser_movido,diretório_de_destino)
                    break
                    
            with open(fr'{path}\\Registro meta 1.1.txt', 'a') as file:
                file.write(f"O arquivo já existia na pasta mas foi adicionado este com o sufixo ({n})")
                file.write("\n\n\n")
            
        else:
            shutil.move(endereco_do_arquivo_a_ser_movido,diretório_de_destino)
            
    else:
        with open(fr'{path}\\Registro meta 1.1.txt', 'a') as file:
            file.write("O arquivo o arquivo não foi encontrado ou baixado ou renomeado ou movido. Precisa-se de tratamento manual :/")
            file.write("\n\n\n")

### Gerando a lista que irá para o txt

In [5]:
def gera_lista_txt(list_info): #Essa função é realmente... útil?
    
    meta = list_info[4][17:20]
    lista_para_txt = list_info.copy()
    
    lista_para_txt[6] = "Escolas Estaduais"
        
    return lista_para_txt

### Gerando o arquivo de txt com as infos de cada Uregi

In [6]:
def arquivo_txt(infos, nome_do_arquivo, uregi, path):

    # Convertendo a lista em string
    infos_str = '; '.join(infos)
    
    with open(fr'{path}\\Registro meta 1.1.txt', 'a') as file:
        file.write(infos_str)
        file.write("\n")
        file.write(nome_do_arquivo)
        file.write("\n\n")

### Executa a ação

In [7]:
def executa_ação(linha, uregi, directory, path, meta_analisada, diretório_de_destino, linha_limite, original_window, driver, list_info):
    # Clicando na imagem para abrir a aba de download
    driver.find_element(By.XPATH, f"//*[@id='tabela']/tbody/tr[{linha}]/td[8]/a/img").click()
    global scroll
    
    try:
        driver.switch_to.window(driver.window_handles[1])

    except:
        # raise Exception("Não foi possível abrir a aba desse arquivo") # Só pra depuração
        winsound.Beep(1000,1000)
        scroll += driver.find_element(By.XPATH, f"//*[@id='tabela']/tbody/tr[{linha}]").rect['height']
        driver.execute_script(f"window.scrollTo(0,{scroll})") #x,y

        linha = linha + 1
        list_info.clear()

    else:
        driver.find_element(By.LINK_TEXT, "Download").click()

        #time.sleep(2) # Importante pois às vezes a aba fecha antes de iniciar o download por questões de internet mesmo
        driver.close()

        driver.switch_to.window(original_window)

        
        # Irá gerar o txt com as informações caso dê algum problema na automação
        arquivo_txt(gera_lista_txt(list_info), formatar_string(list_info, uregi), uregi, path)
        """
            gera_lista_txt: 
                Retorna a list_info mas com apenas o número da meta e o nome da coluna "ciente" que se quiser
            formatar_string:
                Retorna o nome que o arquivo deve ter conforme o padrão
            uregi:
                O nome da Uregi escolhida que será o nome do arquivo txt
        """


        # Renomeia e move o arquivo pra sua pasta correta
        renomear_e_mover_arquivo(formatar_string(list_info, uregi), pegar_nome_original(directory), list_info[3], path, uregi, diretório_de_destino, list_info[4][17:20])
        """
            formatar_string:
                Retorna o nome que o arquivo deve ter conforme o padrão
            pegar_nome_original:
                Retorna o endereço e nome do arquivo mais recente criado (ou seja, o arquivo das metas baixado)
            list_info[2]:
                É apenas a SRE da cidade da linha analisada
        """


        # Eu peço pra que a tela desça o tamanho da altura da linha analizada para abrir caminho para que a próxima linha possa estar visível
        scroll += driver.find_element(By.XPATH, f"//*[@id='tabela']/tbody/tr[{linha}]").rect['height']
        driver.execute_script(f"window.scrollTo(0,{scroll})")

## Código completo

In [8]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip as pc
from pathlib import Path
import winsound
import time
import os
import shutil


# Variáveis que podem/devem ser alteradas conforme a necessidade

índice_da_uregi = 1
# Coloca-se a posição que a uregi de interesse está na caixa de seleção para escolher 

linha = 1
# É a linha onde o programa vai iniciar o processo (linha = 1 é o padrão, a primeira linha)

linha_limite = 0
# Para quando se deseja que o programa pare de executar em determinada linha (se não for necessário, esta deve ser 0)

meta_analisada = "Projeto 1 - Meta 1.1 - Mapa da Oferta de Alimentos da AF"
"""
Opções:
    Projeto 1 - Meta 1.1 - Mapa da Oferta de Alimentos da AF
"""
#É o filtro para a meta específica que se deseja buscar pelo programa, é o valor das células da coluna 'Meta'

directory = Path(r'C:\Users\1234\Downloads') 
# Endereço da pasta onde os arquivos são baixados versão módulo 'os'
path = 'C:\\Users\\1234\\Downloads\\' 
# Endereço da pasta onde os arquivos são baixados versão string
# Lembrar de sempre alterar o caminho do directory e path juntos.

visita_desejada = "todas" 
#As opções são: "todas"; "" pra pegar só as que não colocaram o núm da visita; "1 a 6" o número da visita mesmo

usuario = "61509"
senha = "102492"
#Dados para entrada na intranet



# Inciando a seção e abrindo a tabela da uregi escolhida
driver = webdriver.Chrome()
driver.maximize_window()
original_window = driver.current_window_handle


#Coisa nova porque mudaram o sistema
driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi_teste")

driver.find_element(By.XPATH, f"//*[@id='cliente_usuario_login']").send_keys(usuario)
driver.find_element(By.XPATH, f"//*[@id='loginForm']/p[2]/table/tbody/tr[2]/td[2]/input[1]").send_keys(senha)
driver.find_element(By.XPATH, f"//*[@id='enviar']").click()


# Voltando à normalidade
driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi_teste")

select_element = driver.find_element(By.ID,'mun')
select_object = Select(select_element)
select_object.select_by_index(índice_da_uregi)

uregi = select_object.first_selected_option.text[12:] #!!!

diretório_de_destino = path + "Mapa da Oferta 1.1\\" + uregi
#diretório_de_destino = path + "Evidencias das Uregis - Transição\\" + uregi[12:] + "\\" + meta
#É a pasta para onde vão os arquivos baixados e renomeados

search_button = driver.find_element(By.ID, "ok")
search_button.click()


#Filtrando pela meta analisada
pc.copy(meta_analisada)
driver.find_element(By.XPATH, f"//*[@id='searchFilter']").send_keys(Keys.CONTROL, 'v')


#Colocando a uregi como título do arquivo txt
with open(fr'{path}\\Registro meta 1.1.txt', 'a') as file:
    file.write(uregi + ":")
    file.write("\n\n")


list_info = [] # lista que conterá todas as informações de cada linha da tabela
scroll = 3.5 * driver.find_element(By.XPATH, f"/html/body/div[1]").rect['height'] # Armazena a soma das alturas das linhas da tabela
driver.execute_script(f"window.scrollTo(0,{scroll})")


while True:

    #try:
        
    #Parando de pegar assim que ele encontra uma data na coluna de 'Enviado SEI'
    fim_dos_novos = driver.find_element(By.XPATH, f"//*[@id='tabela']/tbody/tr[{linha}]/td[9]").text
    if(len(fim_dos_novos) == 10):
        winsound.Beep(600,500)
        driver.back()
        
        índice_da_uregi += 1
        
        if(índice_da_uregi == 32):
            winsound.Beep(500,600)
            raise Exception(f"O programa terminou de pegar todos os itens novos")
        
        select_element = driver.find_element(By.ID,'mun')
        select_object = Select(select_element)
        select_object.select_by_index(índice_da_uregi)
        
        uregi = select_object.first_selected_option.text[12:] #!!!
        
        diretório_de_destino = path + "Mapa da Oferta 1.1\\" + uregi
        
        search_button = driver.find_element(By.ID, "ok")
        search_button.click()
        
        driver.find_element(By.XPATH, f"//*[@id='searchFilter']").send_keys(Keys.CONTROL, 'v')
        
        #Colocando a uregi como título do arquivo txt
        with open(fr'{path}\\Registro meta 1.1.txt', 'a') as file:
            file.write("\n\n\n")
            file.write(uregi+ ":")
            file.write("\n\n")
        
        linha = 1
        scroll = 3.5 * driver.find_element(By.XPATH, f"/html/body/div[1]").rect['height'] # Armazena a soma das alturas das linhas da tabela
        driver.execute_script(f"window.scrollTo(0,{scroll})")

        winsound.Beep(300,500)


    #Pegando as informações de cada linha e colocando em uma lista
    list_info.append(uregi) #Agora com Uregi em cada linha
    list_info.append(f"Índice: {linha}")
    for i in range(2,8):
        list_info.append(driver.find_element(By.XPATH, f"//*[@id='tabela']/tbody/tr[{linha}]/td[{i}]").text)
    
    executa_ação(linha, uregi, directory, path, meta_analisada, diretório_de_destino, linha_limite, original_window, driver, list_info)
    
    linha = linha + 1
    list_info.clear() #limpa a lista para o próximo turno
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='tabela']/tbody/tr[1]/td[2]"}
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0117B8F3+2406643]
	Ordinal0 [0x0110AF31+1945393]
	Ordinal0 [0x00FFC748+837448]
	Ordinal0 [0x010292E0+1020640]
	Ordinal0 [0x0102957B+1021307]
	Ordinal0 [0x01056372+1205106]
	Ordinal0 [0x010442C4+1131204]
	Ordinal0 [0x01054682+1197698]
	Ordinal0 [0x01044096+1130646]
	Ordinal0 [0x0101E636+976438]
	Ordinal0 [0x0101F546+980294]
	GetHandleVerifier [0x013E9612+2498066]
	GetHandleVerifier [0x013DC920+2445600]
	GetHandleVerifier [0x01214F2A+579370]
	GetHandleVerifier [0x01213D36+574774]
	Ordinal0 [0x01111C0B+1973259]
	Ordinal0 [0x01116688+1992328]
	Ordinal0 [0x01116775+1992565]
	Ordinal0 [0x0111F8D1+2029777]
	BaseThreadInitThunk [0x74E56A14+36]
	RtlInitializeExceptionChain [0x7771A9FF+143]
	RtlInitializeExceptionChain [0x7771A9CA+90]
